<a href="https://colab.research.google.com/github/veydantkatyal/second-brain-ai/blob/main/second_brain_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TRAINING PIPELINE**

**INSTALLATION**

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer comet_ml==3.48.1
!pip install --no-deps unsloth
!pip install unsloth comet_ml==3.48.1

**PREPARE ENVIRONMENT**

In [ ]:
import os
from getpass import getpass

hf_token = getpass("Enter your Hugging Face token. Press Enter to skip: ")
enable_hf = bool(hf_token)
print(f"Is Hugging Face enabled? '{enable_hf}'")

comet_api_key = getpass("Enter your Comet API key. Press Enter to skip: ")
enable_comet = bool(comet_api_key)
comet_project_name = "second-brain-course"
print(f"Is Comet enabled? '{enable_comet}'")

if enable_hf:
    os.environ["HF_TOKEN"] = hf_token
if enable_comet:
    os.environ["COMET_API_KEY"] = comet_api_key
    os.environ["COMET_PROJECT_NAME"] = comet_project_name

Enter your Hugging Face token. Press Enter to skip: ··········
Is Hugging Face enabled? 'True'
Enter your Comet API key. Press Enter to skip: ··········
Is Comet enabled? 'True'


In [ ]:
dataset_id = (
    input(
        "Enter your Hugging Face dataset_id: "
    )
    or "pauliusztin/second_brain_course_summarization_task"
)
print(f"{dataset_id=}")

Enter your Hugging Face dataset_id: 
dataset_id='pauliusztin/second_brain_course_summarization_task'


In [ ]:
import os
import subprocess

max_seq_length = 4096
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)

# Get the active GPU name
try:
    # Run nvidia-smi command to get GPU information
    nvidia_smi_output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader']).decode('utf-8')
    # Extract the active GPU name from the output
    active_gpu_name = nvidia_smi_output.strip()
except FileNotFoundError:
    # If nvidia-smi is not found, set active_gpu_name to None
    active_gpu_name = None

if active_gpu_name and "L4" in active_gpu_name:
    load_in_4bit = False  # Use 4bit quantization to reduce memory usage.
    max_steps = 250  # Reduce training steps to avoiding waiting too long.
else:
    raise ValueError("No Nvidia GPU found or it's not an L4.")  # More specific error message

print("--- Parameters ---")
print(f"{max_steps=}")
print(f"{load_in_4bit=}")
print(f"{dtype=}")

--- Parameters ---
max_steps=250
load_in_4bit=False
dtype=None


**LOAD LLM USING UNSLOTH**

In [ ]:
from unsloth import FastLanguageModel

base_model = "Meta-Llama-3.1-8B-Instruct"  # or unsloth/Qwen2.5-7B-Instruct
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=f"unsloth/{base_model}",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2025.3.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### **Data Preparation**

We now use the Alpaca format to map the instruct dataset into input prompts.

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

In [ ]:
from datasets import load_dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["instruction"]
    outputs = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN

        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
dataset = load_dataset(dataset_id)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

README.md:   0%|          | 0.00/529 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/955k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/575 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/72 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

<a name="Train"></a>
### **Train the model**
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs=1,  # Set this for 1 full training run, while commenting out 'max_steps'.
        max_steps=max_steps,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="comet_ml" if enable_comet else "none",
    ),
)

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/575 [00:00<?, ? examples/s]

Packing train dataset (num_proc=2):   0%|          | 0/575 [00:00<?, ? examples/s]

**show current memory stats**


In [ ]:
!pip install torch
import torch
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
15.158 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 501 | Num Epochs = 5 | Total steps = 250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,072,204,288 (0.52% trained)
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, keras, tensorflow, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/veydantkatyal/second-brain-course/f66a1a10638249838342acd643767c57

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.480100
2,1.522800
3,1.590800
4,1.607400
5,1.593700
6,1.588200
7,1.595500
8,1.466100
9,1.353200
10,1.549000


**show final memory and stats**

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

9515.3816 seconds used for training.
158.59 minutes used for training.
Peak reserved memory = 18.725 GB.
Peak reserved memory for training = 3.567 GB.
Peak reserved memory % of max memory = 84.495 %.
Peak reserved memory for training % of max memory = 16.096 %.


# **INFERENCE**

Let's run the model! You can change the instruction and input - leave the output blank!

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
text_streamer = TextStreamer(tokenizer)


def generate_text(
    instruction, streaming: bool = True, trim_input_message: bool = False
):
    message = alpaca_prompt.format(
        instruction,
        "",  # output - leave this blank for generation!
    )
    inputs = tokenizer([message], return_tensors="pt").to("cuda")

    if streaming:
        return model.generate(
            **inputs, streamer=text_streamer, max_new_tokens=256, use_cache=True
        )
    else:
        output_tokens = model.generate(**inputs, max_new_tokens=256, use_cache=True)
        output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

        if trim_input_message:
            return output[len(message) :]
        else:
            return output

In [ ]:
generate_text(dataset["validation"][0]["instruction"], streaming=True)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights

### Input:
[![](assets/figures/nvidia.svg) Toronto AI Lab  ](https://research.nvidia.com/labs/toronto-ai/)

# 

Align your Latents:High-Resolution Video Synthesis with Latent Diffusion Models

[Andreas Blattmann1 *,†](https://twitter.com/andi_blatt) [Robin Rombach1 *,†](https://twitter.com/robrombach) [Huan Ling2,3,4 *](https://www.cs.toronto.edu/~linghuan/) [Tim Dockhorn2,3,5 *,†](https://timudk.github.io/) [Seung Wook Kim2,3,4](https://seung-kim.github.io/seungkim/) [Sanja Fidler2,3,4](https://www.cs.toronto.edu/~fidler/) [Karsten Kreis2](https://karste

tensor([[128000,  39314,    374,  ...,    198,  74694, 128009]],
       device='cuda:0')

In [ ]:
generate_text(dataset["validation"][0]["instruction"], streaming=False)

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights\n\n### Input:\n[![](assets/figures/nvidia.svg) Toronto AI Lab  ](https://research.nvidia.com/labs/toronto-ai/)\n\n# \n\nAlign your Latents:High-Resolution Video Synthesis with Latent Diffusion Models\n\n[Andreas Blattmann1 *,†](https://twitter.com/andi_blatt) [Robin Rombach1 *,†](https://twitter.com/robrombach) [Huan Ling2,3,4 *](https://www.cs.toronto.edu/~linghuan/) [Tim Dockhorn2,3,5 *,†](https://timudk.github.io/) [Seung Wook Kim2,3,4](https://seung-kim.github.io/seungkim/) [Sanja Fidler2,3,4](https://www.cs.toronto.edu/~fidler/) [Karsten Kreis2](https://karstenkre


## **Saving Fine-tuned LLM**

The last step is to save the fine-tuned LLM locally

In [ ]:
from huggingface_hub import HfApi

model_name = f"{base_model}-Second-Brain-Summary"
print(f"Model name: {model_name}")
model.save_pretrained_merged(
    model_name,
    tokenizer,
    save_method="merged_16bit",
)  # Local saving

Model name: Meta-Llama-3.1-8B-Instruct-Second-Brain-Summary
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 30.49 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:41<00:00,  1.28s/it]


Unsloth: Saving tokenizer... Done.
Done.
